# Worldclim pre-process to arctic

The originally downloaded worldclim files are global in extent, so here we pre-process to reduce the data to just the area we are interested in, the arctic in this case.

This step will open an extent `.tif` and a bunch of Worldclim `.zip` files. It will crop to the extents of the `.tif` (plus a buffer; default to 30px) and reproject. The result will be in memory and then you have to save it out to an explicit location. Here it makes sense to save the outputs with a numeric indicator in the file name so that we can tell what processing step we are at in the pipeline...

For example the input to this step is from the raw download data directory and the output is saved labeled with 02 in the file name.

In [ ]:
from pathlib import Path

from temds.datasources import worldclim

In [ ]:
out_srs = 'EPSG:6931'
extent_file = 'working/aoi_5km_buffer_6931.tiff' # Need to re-arrange the aoi stuff so it should end up in a folder labeled 01

In [ ]:
# config = 'my-config.yml'
# wc_test = subprograms.download('worldclim', config=config)

# Takes about 20 minutues on a 8 core 32GBmemory machachine.

# This will create an object that has all the data in memory that is cropped
# and re-projected to the extent of the AOI.
wc = worldclim.WorldClim(
    'working/download/worldclim', # This is processing step "1": raw download.
    extent_file,
    verbose = True,
    cleanup_uncompressed = False
)


In [ ]:
wc.dataset

In [ ]:
# Think we fixed this bug, see datasources/base.py
!mkdir working/02-arctic/worldclim

In [ ]:
wc.save('working/02-arctic/worldclim/worldclim-arctic-buffered-better.nc', overwrite=True)

